## Compare results for multiple cosmologies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    val_dr=dict_val['hist_val'].copy()
    val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
    
    sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2
    
    size=len(dict_val['hist_val'])
    l1,l2=int(size*0.3),int(size*0.7)
    keys=['chi_1a','chi_1b','chi_1c','chi_1']
    
    for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
        chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})
    
    idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
#     chisqr_dict.update({'chi_sqr1':})
    
    chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
    chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data
    
    idx=60
    spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})
    
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
    
    return chisqr_dict

In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(10000, 128, 128)


In [7]:
df_runs=pd.DataFrame([])

In [8]:
### Load raw images
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:5000][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [24]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
for strg,fldr in zip(['1','2','3','4'],lst):
    key='keras_'+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:5000]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


0.993173 -0.99858624
0.9902537 -0.9988308
0.9918889 -0.9990455
0.98901266 -0.99779767


In [10]:
# ### Extract a few images generated by Lbann
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/'
# fldr='20200806_065947_gen_img_exagan/'

lst=['20200806_070111_gen_img_exagan/']
names=['1_b64']

for strg,fldr in zip(names,lst):
    f_strg=parent_dir+fldr+'dump_outs/trainer0/model0/sgd.testing.epoch.*.step.*_gen_img_instance1_activation_output0.npy'
    f_list=glob.glob(f_strg)
    print(f_list)
    key='lbann_'+strg
#     fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':f_strg}
    arr=[np.load(fname)[:,0,:,:] for fname in f_list]
    images=np.vstack(arr)
    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

# print(s_lbann.shape,np.max(s_lbann))


['/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/20200806_070111_gen_img_exagan/dump_outs/trainer0/model0/sgd.testing.epoch.3.step.3_gen_img_instance1_activation_output0.npy', '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/20200806_070111_gen_img_exagan/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy', '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/20200806_070111_gen_img_exagan/dump_outs/trainer0/model0/sgd.testing.epoch.1.step.1_gen_img_instance1_activation_output0.npy', '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/20200806_070111_gen_img_exagan/dump_outs/trainer0/model0/sgd.testing.epoch.2.step.2_gen_img_instance1_activation_output0.npy']


In [11]:
### Extract some LBANN training run data

parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200805_124242_batchsize_64/dump_outs/trainer0/model0/'
lst=[(5,16920),(6,20200),(7,23680),(4,14720)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key='lbann_train_{0}-{1}'.format(epoch,step)
    fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_img']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

# print(s_lbann.shape,np.max(s_lbann))



{'label': 'lbann_train_5-16920', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200805_124242_batchsize_64/dump_outs/trainer0/model0/sgd.training.epoch.5.step.16920_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_6-20200', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200805_124242_batchsize_64/dump_outs/trainer0/model0/sgd.training.epoch.6.step.20200_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_7-23680', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200805_124242_batchsize_64/dump_outs/trainer0/model0/sgd.training.epoch.7.step.23680_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_4-14720', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200805_124242_batchsize_64/dump_outs/trainer0/model0/sgd.training.epoch.4.step.14720_gen_img_instance1_activation_output0.npy', 'num_img': 64}


In [12]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'fname', 'hist_bin_centers', 'hist_err',
       'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')

In [13]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img']
df_runs=df_runs.reindex(columns=col_list)

In [14]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0009205326193552958, 0.0006649037769392328,...","[1.11815584318565, 0.9598541797164788, 0.25895...","[74699.29979158258, 647.5545866649256, 321.540...","[199584026.1747691, 88260.19874135335, 61820.3...",0.000283,0.000005,0.000044,0.000234,0.000001,1.420067,0.000242,5.151768e+01,5000.0
1,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0003860635648438461, 0.0006066713155528804,...","[1.859936853190394, 0.1709920956459192, 0.0, 0...","[68618.96545499969, 242.4346881635553, 118.671...","[403827657.5420022, 36477.897519639286, 25868....",2.362336,2.054404,0.295176,0.012757,1.288764,0.169671,138.811219,7.260741e+07,5000.0
2,keras_2,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00040630541763438083, 0.0006384799413335293...","[1.8601615807440373, 0.17063895234770396, 0.0,...","[73033.43055837418, 271.00455843104015, 133.56...","[404325893.55109966, 37649.31830663319, 26671....",2.363214,2.055282,0.295176,0.012757,1.289655,0.178567,136.592758,7.077349e+07,5000.0
3,keras_3,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0004038098214755873, 0.000634558290230927, ...","[1.8587204894443887, 0.17290352438765635, 0.0,...","[74619.32613822428, 247.61748261687273, 125.00...","[406205012.098721, 35852.65532568596, 25296.11...",2.357587,2.049655,0.295176,0.012757,1.283952,0.177470,129.314700,6.749295e+07,5000.0
4,keras_4,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00038890260605150206, 0.0006111326660174179...","[1.8604012413897788, 0.17026234276193208, 0.0,...","[68509.68057374754, 254.3677519579289, 123.115...","[403184653.41350955, 37790.05810079437, 26644....",2.364152,2.056219,0.295176,0.012757,1.290604,0.170919,135.560149,7.705924e+07,5000.0
5,lbann_1_b64,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0010484267450649308, 0.0005757154961456638,...","[0.9969888372348871, 1.0380761091902764, 0.323...","[86409.73383888874, 654.4504741474568, 340.177...","[189146114.99184597, 106339.7244207854, 74306....",0.166337,0.126186,0.025993,0.014158,0.031217,1.512870,0.909840,4.071692e+05,10023.0
6,lbann_train_5-16920,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.01003667361896315, 0.006921975042600593, 0....","[1.1117383568352455, 0.9519060566783413, 0.267...","[767536.0443941664, 5301.743320708245, 3077.67...","[196868850.5717702, 76087.47823429108, 62401.1...",0.011120,0.004860,0.002595,0.003664,0.000425,13.805922,0.301855,1.806022e+03,64.0
7,lbann_train_6-20200,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00738717230113852, 0.005790362668941354, 0....","[1.0952647654650873, 0.9880045667608788, 0.270...","[560179.4771357133, 6146.665142098727, 2613.98...","[197004902.21009278, 81640.11047388613, 57097....",0.015399,0.002233,0.002313,0.010853,0.001563,12.018437,0.286725,1.814452e+03,64.0
8,lbann_train_7-23680,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.011676612004995502, 0.008953113617724913, 0...","[1.0816656975036032, 0.9917427614706933, 0.278...","[843656.343191659, 6165.74442869579, 3469.2087...","[195719296.9659171, 96669.26259374619, 64398.4...",0.016337,0.007474,0.004379,0.004484,0.003052,14.555828,0.147440,6.367050e+02,64.0
9,lbann_train_4-14720,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.008404312139310881, 0.006163919717285438, 0...","[1.0996150402775

### Compare chi-squares

In [21]:
chi_keys=['label','chi_1','chi_spec1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'num_img','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,num_img,fname
0,raw,0.000283,0.000242,0.000005,0.000044,0.000234,0.000001,1.420067,5.151768e+01,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_1,2.362336,138.811219,2.054404,0.295176,0.012757,1.288764,0.169671,7.260741e+07,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_2,2.363214,136.592758,2.055282,0.295176,0.012757,1.289655,0.178567,7.077349e+07,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_3,2.357587,129.314700,2.049655,0.295176,0.012757,1.283952,0.177470,6.749295e+07,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_4,2.364152,135.560149,2.056219,0.295176,0.012757,1.290604,0.170919,7.705924e+07,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,lbann_1_b64,0.166337,0.909840,0.126186,0.025993,0.014158,0.031217,1.512870,4.071692e+05,10023.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,lbann_train_5-16920,0.011120,0.301855,0.004860,0.002595,0.003664,0.000425,13.805922,1.806022e+03,64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,lbann_train_6-20200,0.015399,0.286725,0.002233,0.002313,0.010853,0.001563,12.018437,1.814452e+03,64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,lbann_train_7-23680,0.016337,0.147440,0.007474,0.004379,0.004484,0.003052,14.555828,6.367050e+02,64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,lbann_train_4-14720,0.015372,0.205760,0.006304,0.002103,0.006966,0.001102,12.496350,1.193090e+03,64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...


In [17]:

def f_plot_hist_spec_best(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label=row.label
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   

    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')


f_plot_hist_spec_best(df_runs,dict_val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:

def f_plot_hist_spec_best(df,dict_bkg,plot_type):

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
#     ax1.set_xscale('symlog',linthreshx=50)
#     ax1.set_yscale('log')
#     ax2.set_yscale('log')


f_plot_hist_spec_best(df_runs,dict_val,'hist')
f_plot_hist_spec_best(df_runs,dict_val,'spec')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
def f_widget_compare(sample_names,sample_dict,Fig_type='pixel',rescale=True,log_scale=True,bins=25,mode='avg',normalize=True,bkgnd=[]):
    '''
    Module to make widget plots for pixel intensity or spectrum comparison for multiple sample sets
    '''
    
    ### Crop out large pixel values
#     for key in sample_names:
#         print(sample_dict[key].shape)
#         sample_dict[key]=np.array([arr for arr in sample_dict[key] if np.max(arr)<=0.994])
#         print(sample_dict[key].shape)
    
    img_list=[sample_dict[key] for key in sample_names]
    label_list=list(sample_names)
    
    if rescale: 
        for count,img in enumerate(img_list):
            img_list[count]=f_invtransform(img)
        if len(bkgnd): bkgnd=f_invtransform(bkgnd)
#         hist_range=(0,2000)
    else:
        bins=f_transform(bins)
#         hist_range=(-1,0.996)
    assert Fig_type in ['pixel','spectrum'],"Invalid mode %s"%(mode)
    
    if Fig_type=='pixel':
#         f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=hist_range)
        f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=None,bkgnd_arr=bkgnd)

    elif Fig_type=='spectrum':
        f_compare_spectrum(img_lst=img_list,label_lst=label_list,log_scale=log_scale,bkgnd_arr=bkgnd)



In [ ]:
# dict_samples={'raw': s_raw,'keras1':s_keras[0],'keras2':s_keras[1],'s_new1':s_new1,'s_new2':s_new2,'s_new3':s_new3}

bkgnd=s_raw
# bkgnd=[]
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),
                bins=fixed(bins),
                mode=['avg','simple'],bkgnd=fixed(bkgnd))